<a href="https://colab.research.google.com/github/Karthika506/Familial_melanoma/blob/main/Familial_melanoma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_dataset(data_dir):
  images = []
  masks = []
  for filename in os.listdir(data_dir):
    if filename.endswith('.jpg'):
      image_path = os.path.join(data_dir, filename)
      image = cv2.imread(image_path)
      images.append(image)

      mask_filename = filename.replace('.jpg', '_mask.jpg')
      mask_path = os.path.join(data_dir, mask_filename)
      mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
      masks.append(mask)
  return np.array(images), np.array(masks)